In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas
import datetime
from datetime import datetime
import requests
import re
from selenium import webdriver
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


In [2]:
browser = webdriver.Chrome(executable_path='/Users/hugoruizverastegui/Documents/perso github/chromedriver')

In [3]:
# automated install
#browser = webdriver.Chrome(ChromeDriverManager().install())

In [4]:


parameters= [
{
"country": "TH",   
"website": "OIS",   
"url_scrap": (("france","https://www.i-d-s.com/dubai/wine/wines-by-country/france.html?p="),    
("argentina","https://www.i-d-s.com/dubai/wine/wines-by-country/argentina.html?p="),
("australia","https://www.i-d-s.com/dubai/wine/wines-by-country/australia.html?p="),
("chile","https://www.i-d-s.com/dubai/wine/wines-by-country/chile.html?p="),
("new_zealand","https://www.i-d-s.com/dubai/wine/wines-by-country/new-zealand.html?p="),
("south_africa","https://www.i-d-s.com/dubai/wine/wines-by-country/south-africa.html?p="),
("uk","https://www.i-d-s.com/dubai/wine/wines-by-country/united-kingdom.html?p="),
("usa","https://www.i-d-s.com/dubai/wine/wines-by-country/usa.html?p="),
("germany","https://www.i-d-s.com/dubai/wine/wines-by-country/germany.html?p="),
("italy","https://www.i-d-s.com/dubai/wine/wines-by-country/italy.html?p="),
("spain","https://www.i-d-s.com/dubai/wine/wines-by-country/spain.html?p="),
("rest-of-the-world","https://www.i-d-s.com/dubai/wine/wines-by-country/rest-of-the-world.html?p=")
),
    
    
"price_regex":  "finalPrice\" class=\"price-wrapper \"><span class=\"price\">.*?\$(.*?)<",    
#"price_regex":  "<span class=\"price\">\$(.*?)<",
##"data-price-amount=\"(.*?)\"",
"name_regex": "class=\"product-item-link\" href=.*?>(.*?)<",                
"url_regex": "product-item-link\" href=\"(.*?)\"",
"brand_regex": "NA", 
"scrolling_type" : "no scroll",
"pop_up_type" : "have pop up", 
"page_not_found_type" : "yes",
"page_not_found_regex":"find products matching the selection",
"format_change_type" : "yes",
"multiple_pages" : "yes",
"scrapper_activated":"1"
}
    
]

DF_final=pandas.DataFrame()

In [5]:
def pop_up_closing(param,browser_param):
    if param["pop_up_type"]=='no pop up':
        pass
    if param["pop_up_type"]=='have pop up':

        close=browser_param.find_elements_by_class_name("action-close")
        for element in close : 
            try:
                element.click()   
                time.sleep(1)
            except:
                pass

In [15]:
def get_data(param):
    
        l = len(param["url_scrap"])
        i=0
        df_aggregated=pandas.DataFrame()
        nb_issues = 0
        today_date=datetime.today().strftime('%Y-%m-%d')
        
        while i<l:
        #while i<2:
            
            #browser = webdriver.Chrome(executable_path=executable_path)
            #browser = webdriver.Chrome(ChromeDriverManager().install())
            browser=webdriver.Chrome(executable_path='/Users/hugoruizverastegui/Documents/perso github/chromedriver')
            category=param["url_scrap"][i][0]           
            
            url=param["url_scrap"][i][1]
            print(category)
            #print(url)        

            # to change into a parameter
            test_not_found=param["page_not_found_regex"]

            #init
            page_not_found=0

            # different starting point depending on the param page not found 
            if param["page_not_found_type"]=='yes':
                page=1

            if param["page_not_found_type"]=='no':
                page=""


            while page_not_found==0:
                #use of try to handle possible scrapping errors or time outs - https://stackoverflow.com/questions/51943694/execute-when-error-occurs-python
                     
                url_with_page=url+str(page)
                print(url_with_page)     

                browser.get(url_with_page)    

                time.sleep(5)
                #use of the anti pop up function
                pop_up_closing(param,browser)

                #use of the scrolling function
                #scrolling(param,browser)

                #use of format changing
                #format_change(param,browser,page)

                # extract of sourc code 
                time.sleep(2)
                htmlsource = browser.page_source
                

                #use of the commas replace code
                #htmlsource_replaced = replace_commas(param,htmlsource)
                #print(htmlsource)

                time.sleep(2)

                name = re.findall(param["name_regex"], htmlsource)
                print("name: " + str(len(name)))
                print(name)
                
                url_product = re.findall(param["url_regex"], htmlsource)
                print("url_product: " + str(len(url_product)))
                print(url_product)
                
                prices = re.findall(param["price_regex"], htmlsource)
                print("prices: " + str(len(prices)))
                prices = prices[0:len(name)]
                print("reduced prices: " + str(len(prices)))
                print(prices)
                

                
                df=[prices,name,url_product]
                df=pandas.DataFrame(df).transpose()
                df['country']=category
                df_aggregated=df_aggregated.append(df)


                # exit loop after first iteration if there are no pages to go through
                if param["page_not_found_type"]=='no':
                    page_not_found = 1

                if param["page_not_found_type"]=='yes':
                    page_not_found = len(re.findall(test_not_found, htmlsource))
                    page=page+1
                    # only keep the output if there is no shift 


            i=i+1
            browser.close()
             
            print("---------------")
        
        #define columns names

        return df_aggregated





In [16]:
df_wine_diplo=get_data(parameters[0])

france
https://www.i-d-s.com/dubai/wine/wines-by-country/france.html?p=1
name: 12
['M.Chapoutier, Rasteau " Les Gadilles"', "M.Chapoutier, Luberon 'La Ciboise' Rouge", 'Vranken Brut Nature VDC with Gift Box', 'Veuve Clicquot Yellow Label Brut', 'Moët &amp; Chandon Brut Impérial', 'M.Chapoutier, Costières-de-Nîmes, Collection Bio “Areca” &amp; “Rouge” ', '2012 Château La Mission Haut-Brion, Grand Cru Classé de Graves, Pessac-Léognan', '2016 Château Talbot 4ème Cru Classé, Saint-Julien', 'Château Moncontour Vouvray Brut', '2017 Château Talbot 4ème Cru Classe, Saint-Julien', '2016 Chateau Pichon Comtesse Lalande, Grand Cr', '2015 Chateau Les Ormes de Pez Cru Bourgeois Saint Estephe']
url_product: 12
['https://www.i-d-s.com/dubai/m-chapoutier-rasteau-fr1281.html', 'https://www.i-d-s.com/dubai/m-chapoutier-luberon-la-ciboise-rouge-fr1279.html', 'https://www.i-d-s.com/dubai/vranken-brut-nature-vdc-with-gift-box-cp4019.html', 'https://www.i-d-s.com/dubai/veuve-clicquot-yellow-label-brut-cp101

/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_31041/3340895757.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


name: 12
['2017 Chateau Léoville Barton, Saint -Julien', 'Maison Antoine Geoffroy, Beaune, Puligny Montrachet 1Er Cru Les Combettes ', 'Maison Antoine Geoffroy, Beaune, Gevrey Chambertin Vieilles Vignes ', 'Domaine Daniel-Etienne Defaix, Chablis Grand Cru Blanchot ', 'Domaine Daniel-Etienne Defaix, Chablis Vieilles Vignes', '2016 Château Duhart-Milon Rothschild, 4eme Cru Classé, Pauillac', '2019 "Brulieres" De Beychevelle, Haut-Medoc', 'Cave de Turckheim Gewurztraminer Reserve', 'Schröder and Schÿler Margaux Private Selection', '2017 Chateau Croix Mouton', "Perle d'Aurore, Crémant de Bourgogne Brut Rosé", 'Miraval, Côtes de Provence Rosé, Magnum']
url_product: 12
['https://www.i-d-s.com/dubai/2017-chateau-l-oville-barton-saint-julien-fw1210.html', 'https://www.i-d-s.com/dubai/maison-antoine-geoffroy-beaune-puligny-montrachet-1er-cru-les-combettes-fr1307.html', 'https://www.i-d-s.com/dubai/maison-antoine-geoffroy-beaune-gevrey-chambertin-vieilles-vignes-fr1308.html', 'https://www.i-d-s.

/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_31041/3340895757.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


name: 12
['Remoissenet Santenay Blanc 2018', 'Château Mont-Redon Châteauneuf-du-Pape Rouge', 'Perrier-Jouët Grand Brut', 'Savigny Les Beaune Antoine Geoffroy', 'Domaine Michel Girault Les Beaux Regards, Sancerre Rouge', 'M.Chapoutier, Côtes-du-Rhône Rouge Adubatio , Collection Bio', "M.Chapoutier, 'Belleruche' Côtes-du-Rhône Rouge", 'M.Chapoutier, Châteauneuf-du-Pape, Collection Bio Rouge', 'Marius, Avenue de la Gare Syrah, IGP Collines Rhodaniennes ', "M.Chapoutier, Cornas 'Les Arènes'", "M.Chapoutier, Saint-Joseph 'Deschants' Rouge", "M.Chapoutier, Châteauneuf-du-Pape 'La Bernardine' Blanc"]
url_product: 12
['https://www.i-d-s.com/dubai/remoissenet-santenay-blanc-2018-fr4429.html', 'https://www.i-d-s.com/dubai/ch-teau-mont-redon-ch-teauneuf-du-pape-rouge-zz18126.html', 'https://www.i-d-s.com/dubai/perrier-jou-t-grand-brut-cp1003.html', 'https://www.i-d-s.com/dubai/savigny-les-beaune-antoine-geoffroy-fr4436.html', 'https://www.i-d-s.com/dubai/domaine-michel-girault-les-beaux-regards-s

...

/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_31041/3340895757.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


name: 1
['Calusari Pinot Noir']
url_product: 1
['https://www.i-d-s.com/dubai/calusari-pinot-noir-ro1001.html']
prices: 1
reduced prices: 1
['8.85']
https://www.i-d-s.com/dubai/wine/wines-by-country/rest-of-the-world.html?p=4


/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_31041/3340895757.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


name: 0
[]
url_product: 0
[]
prices: 0
reduced prices: 0
[]
---------------


/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_31041/3340895757.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


In [27]:
df_wine_diplo

,0,1,2,country,vivino_url,vivino_rating,vivino_name,vivino_price,vivino_country,vivino_region,nb_ratings
0,16.95,"M.Chapoutier, Rasteau "" Les Gadilles""",https://www.i-d-s.com/dubai/m-chapoutier-raste...,france,https://www.vivino.com/FR/en/wines/169803930,3.7,M. Chapoutier Les Gadilles Rasteau 0,15.50,france,rasteau,166
1,9.25,"M.Chapoutier, Luberon 'La Ciboise' Rouge",https://www.i-d-s.com/dubai/m-chapoutier-luber...,france,https://www.vivino.com/FR/en/wines/1471076,3.6,M. Chapoutier Luberon La Ciboise 0,—,france,luberon,5088
2,33.00,Vranken Brut Nature VDC with Gift Box,https://www.i-d-s.com/dubai/vranken-brut-natur...,france,https://www.vivino.com/FR/en/wines/155582683,3.7,Vranken Brut Nature Champagne 0,—,france,champagne,565
3,76.85,Veuve Clicquot Yellow Label Brut,https://www.i-d-s.com/dubai/veuve-clicquot-yel...,france,https://www.vivino.com/FR/en/wines/164942636,4.3,Veuve Clicquot Brut (Carte Jaune) Champagne 0,—,france,champagne,126956
4,63.05,Moët &amp; Chandon Brut Impérial,https://www.i-d-s.com/dubai/mo-t-chandon-brut-...,france,https://www.vivino.com/FR/en/wines/164942645,4.1,Moët \u0026 Chandon Impérial Brut Champagne 0,—,france,champagne,115219
...,...,...,...,...,...,...,...,...,...,...,...
8,10.85,"Vinho Verde Loureiro/Alvarinho, Azevedo, Portugal",https://www.i-d-s.com/dubai/vinho-verde-lourei...,rest-of-the-world,https://www.vivino.com/FR/en/wines/161350291,3.8,Quinta de Azevedo Azevedo Loureiro - Alvarinho 0,—,portugal,vinho-verde,2267
9,8.15,Mateus Rosé,https://www.i-d-s.com/dubai/mateus-ros-pt1002....,rest-of-the-world,https://www.vivino.com/FR/en/wines/164942646,3.7,Mateus The Original Rosé 0,—,portugal,douro,50172
10,8.00,"Terra Franca,Vinho Regional Tinto",https://www.i-d-s.com/dubai/terra-franca-vinho...,rest-of-the-world,https://www.vivino.com/FR/en/wines/3474269,0.0,Ouro Da Terra Vinho Regional Beiras Tinto 0,—,portugal,alentejano,2603
11,8.85,Calusari Pinot Grigio,https://www.i-d-s.com/dubai/calusari-pinot-gri...,rest-of-the-world,https://www.vivino.com/FR/en/wines/3130030,3.7,Călușari Pinot Grigio 0,—,romania,recas,1891


In [36]:
df_wine_diplo.to_csv('test2.csv')

In [37]:
#test
browser=webdriver.Chrome(executable_path='/Users/hugoruizverastegui/Documents/perso github/chromedriver')
browser.get('https://www.i-d-s.com/dubai/wine/wines-by-country/france.html?p=1')  
htmlsource = browser.page_source
htmlsource

In [17]:
df_wine_diplo['vivino_url']=''
df_wine_diplo['vivino_rating']=''

In [18]:
df_wine_diplo_backup=df_wine_diplo

In [19]:
def scrap_feature_vivino(feature_name,feature_regex,vivino_feature_list):
        if len(re.findall(feature_regexp, htmlsource))==0:
            vivino_feature_inter_list=['-']
        else:
            vivino_feature_inter_list=[re.findall(url_regexp, htmlsource)[0]]
            
        vivino_feature_list=vivino_feature_list+vivino_feature_inter_list
    

In [20]:
#param for vivino scrapper
features_vivino=['url','rating']

rating_regexp="\"ratingValue\":(.*?),"
url_regexp='\"Product\",\"\@id\":\"(.*?)\",'
name_regexp="description\":\"Check out this (.*?) on Vivino" 
price_regexp="class=\"wine-price-value\">(.*?)</span>"
region_regexp="wine-regions\/(.*?)\""
country_regexp="wine-countries\/(.*?)\""
nb_ratings_regexp="(\d+) ratings"

In [40]:
# Code to send a wine name to Vivino and scrape the first result


vivino_url_wine=[]
vivino_rating_wine=[]
vivino_name_wine=[]
vivino_price_wine=[]
vivino_country_wine=[]
vivino_region_wine=[]
vivino_nb_ratings_wine=[]

list_wine_search=df_wine_diplo[1]

# open browser
#browser = webdriver.Chrome(ChromeDriverManager().install())
browser=webdriver.Chrome(executable_path='/Users/hugoruizverastegui/Documents/perso github/chromedriver')
browser.get('https://www.vivino.com/FR/en/')
#for each wine name in the list 
for search_txt in list_wine_search[0:10]:
    # write the wine name
    #browser.get('https://www.vivino.com/FR/en/')  
    time.sleep(1)
    try:
        # find the input bar
        element=browser.find_element_by_class_name("searchBar_searchInput__Jje-K")
        element.clear()
        element.send_keys(search_txt)
        element.send_keys(Keys.RETURN)

        # get the first result
        time.sleep(2)

        htmlsource = browser.page_source
        rating_regexp="\"ratingValue\":(.*?),"
        url_regexp='\"Product\",\"\@id\":\"(.*?)\",'
        name_regexp="description\":\"Check out this (.*?) on Vivino" 
        price_regexp="class=\"wine-price-value\">(.*?)</span>"
        region_regexp="wine-regions\/(.*?)\""
        country_regexp="wine-countries\/(.*?)\""
        nb_ratings_regexp="(\d+) ratings"

        time.sleep(1)

        if len(re.findall(url_regexp, htmlsource))==0:
            vivino_url_wine_inter=['-']
        else:
            vivino_url_wine_inter=[re.findall(url_regexp, htmlsource)[0]]
        
        if len(re.findall(rating_regexp, htmlsource))==0:
            vivino_rating_wine_inter= ['-']
        else:
            vivino_rating_wine_inter= [re.findall(rating_regexp, htmlsource)[0]]
        
        if len(re.findall(price_regexp, htmlsource))==0:
            vivino_price_wine_inter= ['-']
        else:
            vivino_price_wine_inter= [re.findall(price_regexp, htmlsource)[0]]
               
        if len(re.findall(name_regexp, htmlsource))==0:
            vivino_name_wine_inter= ['-']
        else:
            vivino_name_wine_inter= [re.findall(name_regexp, htmlsource)[0]]
            
        if len(re.findall(country_regexp, htmlsource))==0:
            vivino_country_wine_inter= ['-']
        else:
            vivino_country_wine_inter= [re.findall(country_regexp, htmlsource)[0]]
        
        
        if len(re.findall(region_regexp, htmlsource))==0:
            vivino_region_wine_inter= ['-']
        else:
            vivino_region_wine_inter= [re.findall(region_regexp, htmlsource)[0]]
        
        if len(re.findall(nb_ratings_regexp, htmlsource))==0:
            vivino_nb_ratings_wine_inter= ['-']
        else:
            vivino_nb_ratings_wine_inter= [re.findall(nb_ratings_regexp, htmlsource)[0]]
        

        vivino_url_wine=vivino_url_wine+vivino_url_wine_inter
        vivino_rating_wine=vivino_rating_wine+vivino_rating_wine_inter
        vivino_name_wine=vivino_name_wine+vivino_name_wine_inter
        vivino_price_wine=vivino_price_wine+vivino_price_wine_inter
        vivino_country_wine=vivino_country_wine+vivino_country_wine_inter
        vivino_region_wine=vivino_region_wine+vivino_region_wine_inter
        vivino_nb_ratings_wine=vivino_nb_ratings_wine+vivino_nb_ratings_wine_inter
        
        time.sleep(1)

    except:
        # so the same thing but regenerate the browser and wait 60s
        print('blocked - waiting 60')
        browser.close()
        time.sleep(60)
        
        #browser = webdriver.Chrome(ChromeDriverManager().install())
        browser=webdriver.Chrome(executable_path='/Users/hugoruizverastegui/Documents/perso github/chromedriver')
        browser.get('https://www.vivino.com/FR/en/')  
        
        element=browser.find_element_by_class_name("searchBar_searchInput__Jje-K")
        element.clear()
        element.send_keys(search_txt)
        element.send_keys(Keys.RETURN)

        # get the first result
        time.sleep(2)

        htmlsource = browser.page_source
        rating_regexp="\"ratingValue\":(.*?),"
        url_regexp='\"Product\",\"\@id\":\"(.*?)\",'
        name_regexp="description\":\"Check out this (.*?) on Vivino" 
        price_regexp="class=\"wine-price-value\">(.*?)<\/span>"
        region_regexp="wine-regions\/(.*?)\""
        country_regexp="wine-countries\/(.*?)\""
        nb_ratings_regexp="(\d+) ratings"

        time.sleep(1)

        if len(re.findall(url_regexp, htmlsource))==0:
            vivino_url_wine_inter=['-']
        else:
            vivino_url_wine_inter=[re.findall(url_regexp, htmlsource)[0]]
        
        if len(re.findall(rating_regexp, htmlsource))==0:
            vivino_rating_wine_inter= ['-']
        else:
            vivino_rating_wine_inter= [re.findall(rating_regexp, htmlsource)[0]]
        
        if len(re.findall(price_regexp, htmlsource))==0:
            vivino_price_wine_inter= ['-']
        else:
            vivino_price_wine_inter= [re.findall(price_regexp, htmlsource)[0]]
               
        if len(re.findall(name_regexp, htmlsource))==0:
            vivino_name_wine_inter= ['-']
        else:
            vivino_name_wine_inter= [re.findall(name_regexp, htmlsource)[0]]
            
        if len(re.findall(country_regexp, htmlsource))==0:
            vivino_country_wine_inter= ['-']
        else:
            vivino_country_wine_inter= [re.findall(country_regexp, htmlsource)[0]]
        
        
        if len(re.findall(region_regexp, htmlsource))==0:
            vivino_region_wine_inter= ['-']
        else:
            vivino_region_wine_inter= [re.findall(region_regexp, htmlsource)[0]]
        
        if len(re.findall(nb_ratings_regexp, htmlsource))==0:
            vivino_nb_ratings_wine_inter= ['-']
        else:
            vivino_nb_ratings_wine_inter= [re.findall(nb_ratings_regexp, htmlsource)[0]]
        

        vivino_url_wine=vivino_url_wine+vivino_url_wine_inter
        vivino_rating_wine=vivino_rating_wine+vivino_rating_wine_inter
        vivino_name_wine=vivino_name_wine+vivino_name_wine_inter
        vivino_price_wine=vivino_price_wine+vivino_price_wine_inter
        vivino_country_wine=vivino_country_wine+vivino_country_wine_inter
        vivino_region_wine=vivino_region_wine+vivino_region_wine_inter
        vivino_nb_ratings_wine=vivino_nb_ratings_wine+vivino_nb_ratings_wine_inter
        
    

/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_31041/1823086508.py:19: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for search_txt in list_wine_search[0:10]:


In [44]:
vivino_name_wine


['M. Chapoutier Les Gadilles Rasteau 0',
 'M. Chapoutier Luberon La Ciboise 0',
 'Vranken Brut Nature Champagne 0',
 'Veuve Clicquot Brut (Carte Jaune) Champagne 0',
 'Moët \\u0026 Chandon Impérial Brut Champagne 0',
 'M. Chapoutier Collection Bio Costières de Nîmes 0',
 'Château La Mission Haut-Brion Pessac-Léognan (Grand Cru Classé de Graves) 0',
 'Château Talbot Saint-Julien (Grand Cru Classé) 0',
 'Château Moncontour Cuvée Prédilection Vouvray Brut 0',
 'Château Talbot Saint-Julien (Grand Cru Classé) 0']

In [41]:
df_wine_diplo['vivino_url']=vivino_url_wine
df_wine_diplo['vivino_rating']=vivino_rating_wine
df_wine_diplo['vivino_name']=vivino_name_wine
df_wine_diplo['vivino_price']=vivino_price_wine
df_wine_diplo['vivino_country']=vivino_country_wine
df_wine_diplo['vivino_region']=vivino_region_wine
df_wine_diplo['nb_ratings']=vivino_nb_ratings_wine

        

ValueError: Length of values (10) does not match length of index (675)

In [36]:
df_wine_diplo

,0,1,2,country,vivino_url,vivino_rating,vivino_name,vivino_price,vivino_country,vivino_region,nb_ratings,ratio
0,16.95,"M.Chapoutier, Rasteau "" Les Gadilles""",https://www.i-d-s.com/dubai/m-chapoutier-raste...,france,https://www.vivino.com/FR/en/wines/169803930,3.7,M. Chapoutier Les Gadilles Rasteau 0,15.50,france,rasteau,166,0
1,9.25,"M.Chapoutier, Luberon 'La Ciboise' Rouge",https://www.i-d-s.com/dubai/m-chapoutier-luber...,france,https://www.vivino.com/FR/en/wines/1471076,3.6,M. Chapoutier Luberon La Ciboise 0,—,france,luberon,5088,86
2,33.00,Vranken Brut Nature VDC with Gift Box,https://www.i-d-s.com/dubai/vranken-brut-natur...,france,https://www.vivino.com/FR/en/wines/155582683,3.7,Vranken Brut Nature Champagne 0,34.10,france,champagne,565,71
3,76.85,Veuve Clicquot Yellow Label Brut,https://www.i-d-s.com/dubai/veuve-clicquot-yel...,france,https://www.vivino.com/FR/en/wines/164942636,4.3,Veuve Clicquot Brut (Carte Jaune) Champagne 0,—,france,champagne,126965,52
4,63.05,Moët &amp; Chandon Brut Impérial,https://www.i-d-s.com/dubai/mo-t-chandon-brut-...,france,https://www.vivino.com/FR/en/wines/164942645,4.1,Moët \u0026 Chandon Impérial Brut Champagne 0,—,france,champagne,115219,57
...,...,...,...,...,...,...,...,...,...,...,...,...
8,10.85,"Vinho Verde Loureiro/Alvarinho, Azevedo, Portugal",https://www.i-d-s.com/dubai/vinho-verde-lourei...,rest-of-the-world,https://www.vivino.com/FR/en/wines/161350291,3.8,Quinta de Azevedo Azevedo Loureiro - Alvarinho 0,—,portugal,vinho-verde,2267,54
9,8.15,Mateus Rosé,https://www.i-d-s.com/dubai/mateus-ros-pt1002....,rest-of-the-world,https://www.vivino.com/FR/en/wines/164942646,3.7,Mateus The Original Rosé 0,—,portugal,douro,50172,59
10,8.00,"Terra Franca,Vinho Regional Tinto",https://www.i-d-s.com/dubai/terra-franca-vinho...,rest-of-the-world,https://www.vivino.com/FR/en/wines/3474269,0.0,Ouro Da Terra Vinho Regional Beiras Tinto 0,—,portugal,alentejano,2603,68
11,8.85,Calusari Pinot Grigio,https://www.i-d-s.com/dubai/calusari-pinot-gri...,rest-of-the-world,https://www.vivino.com/FR/en/wines/3130030,3.7,Călușari Pinot Grigio 0,—,romania,recas,1891,86


In [30]:
#check name distance 
import fuzzywuzzy
from fuzzywuzzy import fuzz

def distance_name(name1, name2):
    return fuzz.ratio(name1.lower(),name2.lower())

In [31]:
df_wine_diplo['ratio']=df_wine_diplo.apply(lambda x: distance_name(x[1], x['vivino_name']), axis=1)

In [40]:
df_wine_diplo[df_wine_diplo['ratio']<=50]

,0,1,2,country,vivino_url,vivino_rating,vivino_name,vivino_price,vivino_country,vivino_region,nb_ratings,ratio
0,16.95,"M.Chapoutier, Rasteau "" Les Gadilles""",https://www.i-d-s.com/dubai/m-chapoutier-raste...,france,-,-,-,-,-,-,-,0
2,45.75,Savigny Les Beaune Antoine Geoffroy,https://www.i-d-s.com/dubai/savigny-les-beaune...,france,https://www.vivino.com/FR/en/wines/4789464,4.0,Domaine Antoine Olivier Savigny-Les-Beaune Pre...,—,france,savigny-les-beaune-les-peuillets,129,39
3,21.25,"Domaine Trénel, Climat Fleurie 'La Madone' (Si...",https://www.i-d-s.com/dubai/domaine-tr-nel-fle...,france,https://www.vivino.com/FR/en/wines/4833964,3.7,Jean-Marc Despres Domaine de la Madone Fleurie 0,—,france,fleurie,1140,46
4,13.65,"Domaine Trénel, Beaujolais Organic "" Rouge""",https://www.i-d-s.com/dubai/domaine-tr-nel-bea...,france,https://www.vivino.com/FR/en/wines/3582444,3.7,Martine et Pierre Marie Chermette - Domaine du...,15.10,france,beaujolais,3731,43
7,22.95,"Domaine Trénel, Viré Clessé Chardonnay, Vin de...",https://www.i-d-s.com/dubai/domaine-tr-nel-vir...,france,https://www.vivino.com/FR/en/wines/144587348,4.0,Domaine de l'Epinet Gramont Chardonnay 0,—,france,vire-clesse,206,50
10,45.85,Boutinot Cairanne 'Les Six',https://www.i-d-s.com/dubai/boutinot-cairanne-...,france,-,-,-,-,-,-,-,0
3,31.75,"""M"" De Minuty Magnum (1.5L)",https://www.i-d-s.com/dubai/m-de-minuty-magnum...,france,https://www.vivino.com/FR/en/wines/2191238,3.9,Minuty M Rosé 0,16.00,france,cotes-de-provence,40256,43
0,22.85,Macon Rouge Maison Antoine Geoffroy,https://www.i-d-s.com/dubai/macon-rouge-maison...,france,https://www.vivino.com/FR/en/wines/147009133,3.8,Maison Guillot-Broux Mâcon Rouge 0,—,france,macon,301,46
5,295.95,"Dom Pérignon Vintage, in individual Gift Box",https://www.i-d-s.com/dubai/dom-p-rignon-vinta...,france,https://www.vivino.com/FR/en/wines/1540082,4.6,Dom Pérignon Brut Champagne 0,250.00,france,champagne,148789,49
1,99.60,"Château Minuty ""281"" Rosé",https://www.i-d-s.com/dubai/ch-teau-minuty-281...,france,https://www.vivino.com/FR/en/wines/1473981,4.1,Château d'Esclans Whispering Angel Rosé 0,19.95,france,cotes-de-provence,79131,48


In [ ]:
# for the wine with ratio <50 they are scrapped a second time

In [32]:
df_wine_diplo.to_csv('wine_diplo_2023_28Aug.csv')